In [9]:
import pandas as pd
import os
import cv2
import matplotlib.pyplot as plt
import shutil
import glob
import concurrent.futures

DATASET FOLDERS AND SUBFOLDERS GENERATION

In [ ]:
''' Loads words from text file of listed words separated by /n'''

def load_words(pth : str) -> list[str]:
    words=[]
    with open(pth,'r',encoding='utf-8') as words_text:
        for line in words_text:
           indexed_words=line.strip().split('. ',1) #indexed_words is raw data with number
           word=indexed_words[1]          #separating number and words
           words.append(word)
    return words # returns a useable words list from text file

In [ ]:
''' Makes folders and subfolders from a parent folder's list and child folders list'''

def make_folders(location:str,prt_list,chld_list): # (destination path ,parent directory , child directory)
    for folder in prt_list:
        try:
            if not os.path.exists(os.path.join(location,str(folder))):
                os.mkdir(os.path.join(location,str(folder)))
            else:
                print(f"parent foldar already exist{os.path.join(location,str(folder))}")


            for ch_folder in chld_list:
                    
                try:
                    if not os.path.exists(os.path.join(location,str(folder),str(ch_folder))):
                        os.mkdir(os.path.join(location,str(folder),str(ch_folder)))
                    else:
                        print(f"child foldar already exist{os.path.join(location,str(folder),str(ch_folder))}")        
                except:
                    print(f"invalid syntax in child{os.path.join(location,str(folder),str(ch_folder))}")         

        except:
            print(f"invalid syntax in parent{os.path.join(location,str(folder))}")
        
            


In [ ]:
total_speaker=150
word_text_path= r'.\Raw_Data\uniquelist.txt' # path for file that contains words list
datast_path=r'.\Dataset' #path where we want to save the data set

parent_folders=[f"s{i}" for i in range(1,total_speaker+1)] #generating parent folder ie. speaker folder s1,s2,s3,s4

make_folders(datast_path,parent_folders,load_words('Raw_Data/uniquelist.txt'))


ANNOTATION FILE PROCESSING FUNCTIONS

In [3]:
#grabs xl file from the given path
def find_xl(pth):
    all_files=os.listdir(pth)
    xl = [file for file in all_files if file.endswith('.xlsx')]
    xl_loc=os.path.join(pth,xl[0])
    try:
        #print(xl)
        if len(xl)==1:
            return xl_loc
        else:
            print("Too many xl")
            xl=[]
    except:
        print('no xl error')

In [ ]:
def read_xl_vids(poth:str) -> list[str]:  #returns annotated vdo list from xl
    df=pd.read_excel(poth)
    col=[i for i in df.columns]
    col=col[2:]
    annotated_vids=[i for i in col if (df[i] == "0::.,0::." ).sum()<50] # checks if there are more than 50 empty entry in the columns, if true discards it
    try:
        return annotated_vids
    except:
        print("no particular annotation")

In [ ]:
''' Converts 0:0:0.0 (H:Min:Sec.Ms) to milliseconds format for checking the annotation entry'''

def tstr2ms(time_string):
    # Split the time string into hours, minutes, seconds, and milliseconds
    time_parts = time_string.split(':')

    # Convert hours, minutes, seconds, and milliseconds to integers (use 0 if missing)
    hours = int(time_parts[0]) if len(time_parts) > 0 and time_parts[1]!="" else 0
    minutes = int(time_parts[1]) if len(time_parts) > 1 and time_parts[1]!="" else 0
    seconds_parts = time_parts[2].split('.')
    seconds = int(seconds_parts[0]) if len(seconds_parts)> 0 and seconds_parts[0]!=""  else 0
    if len(str(seconds_parts[1]))<2:
           milliseconds = (int(seconds_parts[1])*100)  if len(seconds_parts) > 1 and seconds_parts[1]!="" else 0
           #print("in the one digit range")
    elif len(str(seconds_parts[1]))<3:
         milliseconds = (int(seconds_parts[1])*10)  if len(seconds_parts) > 1 and seconds_parts[1]!="" else 0
         #print("in the two digit range")
    else:
        milliseconds = int(seconds_parts[1]) if len(seconds_parts) > 1 and seconds_parts[1]!="" else 0
        #print("in the three digit range")



    # Calculate total milliseconds
    total_milliseconds = (
        hours * 60 * 60 * 1000 +
        minutes * 60 * 1000 +
        seconds * 1000 +
        milliseconds
    )

    return total_milliseconds



In [ ]:
'''   Checks if annotation entries are valid or not and returns start and ending time of words of a particular speaker'''

def chk_annotation(vdi:str)->list[int]:
    annt=[a for a in df[vdi]]
    #print(annt)
    stym=[]
    etym=[]
    
    for k,val in enumerate(annt):
        try:
         stym.append(tstr2ms(val.split(",")[0])) 
         etym.append(tstr2ms(val.split(",")[1]))
         #stym.append(val.split(",")[0]) #uncomment for movie pie
         #etym.append(val.split(",")[1])
         if tstr2ms(val.split(",")[0])>tstr2ms(val.split(",")[1]):
            print(f"error:STRT-TIME > END-TIME in index {k+1}")
         if abs(tstr2ms(val.split(",")[0])-tstr2ms(val.split(",")[1]))<200:
            print(f"error:less than 200ms for word in index {k+1}")
         if abs(tstr2ms(val.split(",")[0])-tstr2ms(val.split(",")[1]))>3000:
            print(f"error:more than 3 sec for word in index {k+1}")
        
        except:
           print(f"error:format doesn't match in index {k+1}")   
      
    return stym,etym
    


VIDEO PROCESSING FUNCTIONS


In [ ]:
'''SAVES IMAGE FRAMES TO DESTANTION IN VERBOSE MODE'''
def save_frame(frame_info):
    frame_count, frame, frame_filename, output_folder = frame_info
    cv2.imwrite(frame_filename, frame, [cv2.IMWRITE_PNG_COMPRESSION, 5])
    shutil.move(frame_filename, output_folder)
    
    print(f"Frame {frame_count} saved to {output_folder}")
    


In [ ]:
'''CROPING IMAGE PROCESSING FUNCTION RETURNS INFORMATION ABOT A FRAME TO SAVE'''
def save_frames(video_path, output_folder, start_time, end_time):
    # Open the video file
    cap = cv2.VideoCapture(video_path)

    # Check if the video opened successfully
    if not cap.isOpened():
        print("Error: Could not open video.")
        return

    # Get frames per second (fps)
    fps = cap.get(cv2.CAP_PROP_FPS)
    # Calculate start and end frame indices
    start_frame = int(start_time/1000 * fps)
    end_frame = int(end_time/1000 * fps)

    # Create output folder if it doesn't exist
    os.makedirs(output_folder, exist_ok=True)

    # Set the video capture position to the start frame
    cap.set(cv2.CAP_PROP_POS_FRAMES, start_frame)

    # Read and save frames within the specified range
    frame_count = start_frame
    i = 1
    _ = [os.remove(file) for file in glob.glob(os.path.join(r"D:\Dataset\buffer", '*.png'))]

    frames_info = []

    while frame_count <= end_frame:
        ret, frame = cap.read()
        #get the width and height
        width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

        if not ret:
            print(f"Error: Could not read frame {frame_count}.")
            break

        # Save the frame as an image in the output folder
        frame_filename = os.path.join(r"D:\Dataset\buffer", f"{i:02d}.png")
        if width != 1280 and height != 720:
            frame = cv2.resize(frame, (1280, 720), interpolation=cv2.INTER_AREA)

        frames_info.append((frame_count, frame.copy(), frame_filename, output_folder))  # Use copy to avoid shared memory issues

        frame_count += 1
        i += 1

    # Release the video capture object
    cap.release()

    # Use ThreadPoolExecutor for parallel processing
    with concurrent.futures.ThreadPoolExecutor() as executor:
        executor.map(save_frame, frames_info)

    return




STEP1: Find XL location and read the XL and find available annotated videos

STEP2: Check the annotations


STEP3: Make input and output folders


STEP4: RUN THE LOOP BOX


In [ ]:

Raw_data_Path = r'.\Raw_Data'
xl_path=find_xl(Raw_data_Path)
df=pd.read_excel(xl_path)
avialable = read_xl_vids(xl_path) ## available is the list of available annotated colums which has less than 50 missing entries

In [ ]:
vid_name="s7.mp4"  ## put the names from the available list
stym,etym=chk_annotation(vid_name)

In [ ]:

video_path= os.path.join(Raw_data_Path,vid_name)  #videos from base location
output_path=r'.\Dataset' #path where you want to save the dataset
speaker_folder='s1'      #name of the speakr folder where you want to save

In [ ]:

for i,word in enumerate(df['words']):
    output_folder=os.path.join(output_path,speaker_folder,word)
    start_time=stym[i]
    end_time=etym[i]
    if not start_time==end_time: ## checks if entry is empty or not
       save_frames(video_path, output_folder, start_time, end_time)

Cleaning The Dataset

In [ ]:
speaker_path = '.\\Dataset\\s132'

In [ ]:
'''Removes subfolders which have less than 5 frames and more than 45 frames'''
for f in os.listdir():
    if len(os.listdir(os.path.join(speaker_path,f)))<5:
        os.rmdir(os.path.join(speaker_path,f))
        print(f"removed : {f}")
        
    if len(os.listdir(os.path.join(speaker_path,f)))>45:
        os.rmdir(os.path.join(speaker_path,f))
        print(f"removed : {f}")
            
        